# Importing Libraries

In [1]:
import pandas as pd
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from sklearn.metrics import multilabel_confusion_matrix

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense ,Dropout
from tensorflow.keras.applications import VGG16, VGG19, ResNet50, ResNet101

2024-03-24 12:03:37.942995: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 12:03:37.943137: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 12:03:38.109367: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Reading Data

In [3]:
complete_data_info = pd.read_csv("/kaggle/input/data/Data_Entry_2017.csv")

# Data Preprocessing

In this the path and file names of each .png image is separated and stored in a two different list and a new dataframe is made with these path and filenames and then it is merged with the complete data file. It is done as there is no path column in the dataset provided and we would need image path later. And then we have dropped unnecessary columns from the the complete_data_info. 

In [4]:
data_base_path = "/kaggle/input/data"

In [5]:
complete_data_imgs_paths = list()
imgs_file_names = list()

for single_img_path in Path(data_base_path).glob("images_*/images/*.png"):
    
    complete_data_imgs_paths.append(str(single_img_path))
    imgs_file_names.append(str(single_img_path.parts[-1]))

In [6]:
complete_data_path_info = pd.DataFrame(data={"Image Index":imgs_file_names,
                                            "Image Path":complete_data_imgs_paths})

In [7]:
complete_data_all_info = complete_data_info.merge(complete_data_path_info,on="Image Index")

In [8]:
complete_data_all_info.drop(complete_data_all_info.columns[2:6],axis=1,inplace=True)

In [9]:
complete_data_all_info.drop(["Unnamed: 11"],axis=1,inplace=True)

# Reading the Testing data

In [10]:
testing_data_info = pd.read_csv("/kaggle/input/data/BBox_List_2017.csv")

# Getting the training data from complete data - testing data

In [11]:
complete_data_all_info = complete_data_all_info.set_index("Image Index")

In [12]:
testing_data_path_info = complete_data_all_info.loc[testing_data_info["Image Index"]]

In [13]:
training_data_all_info = complete_data_all_info.drop(index=testing_data_path_info.index)

In [14]:
testing_data_path_info.reset_index(inplace=True)

Adding path info to the testing data and deleting unnecessary coumns from the testing data

In [15]:
testing_data_all_info = testing_data_info.merge(testing_data_path_info,
                                               on="Image Index")

In [16]:
testing_data_all_info.drop(labels=testing_data_all_info.columns[6:9],axis=1,
                          inplace=True)

In [17]:
training_data_all_info.head()

,Finding Labels,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Image Path
Image Index,,,,,,,
00000001_000.png,Cardiomegaly,PA,2682,2749,0.143,0.143,/kaggle/input/data/images_001/images/00000001_...
00000001_001.png,Cardiomegaly|Emphysema,PA,2894,2729,0.143,0.143,/kaggle/input/data/images_001/images/00000001_...
00000001_002.png,Cardiomegaly|Effusion,PA,2500,2048,0.168,0.168,/kaggle/input/data/images_001/images/00000001_...
00000002_000.png,No Finding,PA,2500,2048,0.171,0.171,/kaggle/input/data/images_001/images/00000002_...
00000003_000.png,Hernia,PA,2582,2991,0.143,0.143,/kaggle/input/data/images_001/images/00000003_...


In [18]:
testing_data_all_info.head()

,Image Index,Finding Label,Bbox [x,y,w,h],Finding Labels,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Image Path
0,00013118_008.png,Atelectasis,225.084746,547.019217,86.779661,79.186441,Atelectasis,PA,2992,2991,0.143,0.143,/kaggle/input/data/images_006/images/00013118_...
1,00014716_007.png,Atelectasis,686.101695,131.543498,185.491525,313.491525,Atelectasis|Effusion|Mass,AP,3056,2544,0.139,0.139,/kaggle/input/data/images_007/images/00014716_...
2,00029817_009.png,Atelectasis,221.830508,317.053115,155.118644,216.949153,Atelectasis,AP,3056,2544,0.139,0.139,/kaggle/input/data/images_012/images/00029817_...
3,00014687_001.png,Atelectasis,726.237288,494.951420,141.016949,55.322034,Atelectasis|Cardiomegaly|Consolidation,AP,2500,2048,0.168,0.168,/kaggle/input/data/images_007/images/00014687_...
4,00017877_001.png,Atelectasis,660.067797,569.780787,200.677966,78.101695,Atelectasis,AP,2500,2048,0.168,0.168,/kaggle/input/data/images_008/images/00017877_...


# Getting Unique Diseases

Using the map(), the list of Finding Labels is traversed and the diseases are split where there is a '|' and then it is merged on the list and we got a list of uniques diseaes as a result.  

In [19]:
training_data_all_info["Finding Labels"] = training_data_all_info["Finding Labels"].map(lambda x: x.split("|"))
testing_data_all_info["Finding Labels"] = testing_data_all_info["Finding Labels"].map(lambda x: x.split("|"))

In [20]:
merged_list = list()

for single_list in training_data_all_info["Finding Labels"]:
    
    merged_list = merged_list + single_list

unique_diseases = set(merged_list)
unique_diseases = list(unique_diseases)
unique_diseases.remove("No Finding")
disease2idx = dict(zip(unique_diseases,range(0,len(unique_diseases))))

In [21]:
disease2idx

{'Pneumothorax': 0,
 'Edema': 1,
 'Fibrosis': 2,
 'Infiltration': 3,
 'Mass': 4,
 'Consolidation': 5,
 'Pneumonia': 6,
 'Cardiomegaly': 7,
 'Atelectasis': 8,
 'Pleural_Thickening': 9,
 'Hernia': 10,
 'Effusion': 11,
 'Emphysema': 12,
 'Nodule': 13}

# Multihot Encoding of the Diseases

In this initially the list has zeros and if the image has has diseases the that index will be changed to one. In this we will create a multihot encoded vector for both training and testing data. This is done by ThreadPoolExecutor to manage and create threads. It is done using map(), it maps the method and iterables together immediately and will raise an exception concurrent.futures.TimeoutError if it fails to do so within the timeout limit.

In [22]:
def map_diseases(disease_list):
    
    all_zeros = np.zeros(len(disease2idx),)
    
    for single_disease in disease_list:
        
        if single_disease != "No Finding":
            all_zeros[disease2idx[single_disease]] = 1
        
    return all_zeros

In [23]:
with ThreadPoolExecutor(max_workers=os.cpu_count()) as pool:
    
    multi_hot_encoded_Y_train = np.array(list(pool.map(map_diseases,training_data_all_info["Finding Labels"])))
    multi_hot_encoded_Y_test = np.array(list(pool.map(map_diseases,testing_data_all_info["Finding Labels"])))

In [24]:
training_data_all_info.reset_index(inplace=True)

Training and Testing data are stored in .csv files after all the preprocessing.

In [25]:
training_data_all_info.to_csv("training_data.csv",index=False)
testing_data_all_info.to_csv("testing_data.csv",index=False)

In custom_training_data_generator(), the minibatch from the training is selected at random and then it 

In [26]:
def custom_training_data_generator(training_data_all_info,multi_hot_encoded_Y_train,mb_size):
    idx = list(training_data_all_info.index)
    np.random.shuffle(idx)
    training_data_all_info=training_data_all_info.iloc[idx]
    multi_hot_encoded_Y_train=multi_hot_encoded_Y_train[idx]
    
    for time_step in range (training_data_all_info.shape[0]//mb_size):
        X_train_mb=list()
        
        for single_img_path in training_data_all_info.iloc[time_step*mb_size:(time_step+1)*mb_size]["Image Path"]:
#             resized_single_img = cv2.resize(Image.open(single_img_path).convert("RGB"),(1024,1024))
    
#             X_train_mb.append(resized_single_img)
             # Open the image using PIL
            pil_img = Image.open(single_img_path).convert("RGB")

            # Convert PIL image to numpy array
            np_img = np.array(pil_img)

            # Resize the numpy array
            resized_single_img = cv2.resize(np_img, (1024, 1024))

            X_train_mb.append(resized_single_img)
            
        X_train_mb=np.array(X_train_mb)
        
        Y_train_mb=multi_hot_encoded_Y_train[time_step*mb_size:(time_step+1)*mb_size]
        
        yield X_train_mb,Y_train_mb

In [27]:
def custom_testing_data_generator(testing_data_all_info, multi_hot_encoded_Y_test, mb_size):
    
    for time_step in range(testing_data_all_info.shape[0]//mb_size):
        
        X_test_mb = list()
        
        for single_img_path in testing_data_all_info.iloc[time_step*mb_size:(time_step+1)*mb_size]["Image Path"]:
            
            resized_single_img =  cv2.resize(np.array(Image.open(single_img_path).convert("RGB")),(1024,1024))
            X_test_mb.append(resized_single_img)
            
        X_test_mb = np.array(X_test_mb)
        Y_test_mb = multi_hot_encoded_Y_test[time_step*mb_size:(time_step+1)*mb_size]
        
        bbox_mb = np.array(testing_data_all_info.iloc[time_step*mb_size:(time_step+1)*mb_size,2:6])
        centroid_mb = bbox_mb[:,0:2] + 0.5*np.concatenate((bbox_mb[:,3:4],bbox_mb[:,2:3]),axis=1)
        
        img_size_mb = np.array(testing_data_all_info.iloc[time_step*mb_size:(time_step+1)*mb_size,8:10])
        ordered_img_size_mb = np.concatenate((img_size_mb[:,1:2],img_size_mb[:,0:1]),axis=1)
        rescaled_centroid_mb = centroid_mb * (32/ordered_img_size_mb)
        
        yield X_test_mb, Y_test_mb, rescaled_centroid_mb

In [28]:
class GlobalLSEPooling2D(tf.keras.layers.Layer):
    
    def __init__(self,r_hyper_param):
        super(GlobalLSEPooling2D, self).__init__()
        self.r = r_hyper_param
        
    def call(self,concatenated_input):
        x_star_per_channel = GlobalMaxPooling2D(keepdims=True)(concatenated_input)
        shifted_pix_values = tf.math.exp(self.r*(concatenated_input - x_star_per_channel))
        avged_output = GlobalAveragePooling2D(keepdims=True)(shifted_pix_values)
        logged_output = (1/self.r)*tf.math.log(avged_output)
        layer_output = x_star_per_channel + logged_output
        
        return layer_output

In [29]:
def chest_x_ray_cnn():
    
    input_to_cnn = Input(shape=(1024,1024,3))
    
    pretrained_vgg16_conv_base = VGG16(include_top=False,
                                   input_shape=(1024,1024,3))
    pretrained_vgg19_conv_base = VGG19(include_top=False,
                                  input_shape=(1024,1024,3))
    pretrained_resnet50_conv_base = ResNet50(include_top=False,
                                        input_shape=(1024,1024,3))
    pretrained_resnet101_conv_base = ResNet101(include_top=False,
                                          input_shape=(1024,1024,3))

    pretrained_vgg16_conv_base.trainable = False
    pretrained_vgg19_conv_base.trainable = False
    pretrained_resnet50_conv_base.trainable = False
    pretrained_resnet101_conv_base.trainable = False
    
    vgg16_out = pretrained_vgg16_conv_base(input_to_cnn)
    vgg19_out = pretrained_vgg19_conv_base(input_to_cnn)
    resnet50_out = pretrained_resnet50_conv_base(input_to_cnn)
    resnet101_out = pretrained_resnet101_conv_base(input_to_cnn)
    
    concatenated_output = Concatenate()([vgg16_out,vgg19_out,
                                      resnet50_out,resnet101_out])
    
    pooled_output = GlobalLSEPooling2D(r_hyper_param=0.9)(concatenated_output)
    flattened_output = Flatten()(pooled_output)
    dropout_output = Dropout(0.5)(flattened_output)
    
    cnn_out = Dense(units=multi_hot_encoded_Y_train.shape[1],
                    activation="sigmoid")(dropout_output)
    
    return Model(inputs=input_to_cnn,outputs=cnn_out)

In [30]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [31]:
# def chest_x_ray_cnn():
#     input_to_cnn = Input(shape=(1024, 1024, 3))

#     pretrained_vgg16_conv_base = VGG16(include_top=False, input_shape=(1024, 1024, 3))
#     pretrained_vgg19_conv_base = VGG19(include_top=False, input_shape=(1024, 1024, 3))
#     pretrained_resnet50_conv_base = ResNet50(include_top=False, input_shape=(1024, 1024, 3))
#     pretrained_resnet101_conv_base = ResNet101(include_top=False, input_shape=(1024, 1024, 3))

#     pretrained_vgg16_conv_base.trainable = False
#     pretrained_vgg19_conv_base.trainable = False
#     pretrained_resnet50_conv_base.trainable = False
#     pretrained_resnet101_conv_base.trainable = False

#     vgg16_out = pretrained_vgg16_conv_base(input_to_cnn)
#     vgg19_out = pretrained_vgg19_conv_base(input_to_cnn)
#     resnet50_out = pretrained_resnet50_conv_base(input_to_cnn)
#     resnet101_out = pretrained_resnet101_conv_base(input_to_cnn)

#     concatenated_output = Concatenate()([vgg16_out, vgg19_out, resnet50_out, resnet101_out])

#     pooled_output = GlobalAveragePooling2D()(concatenated_output)
#     dense_output = Dense(units=512, activation='relu')(pooled_output)
#     dropout_output = Dropout(0.5)(dense_output)  # Adding Dropout for regularization
#     cnn_out = Dense(units=multi_hot_encoded_Y_train.shape[1], activation='sigmoid')(dropout_output)

#     model = Model(inputs=input_to_cnn, outputs=cnn_out)

#     # Compile the model
#     optimizer = Adam(lr=0.001)  # Starting with a lower learning rate
#     model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

#     return model

# # Define a learning rate reduction callback
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# # Usage:
# model = chest_x_ray_cnn()
# # model.fit(custom_training_data_generator, epochs=epochs, validation_data=validation_generator, callbacks=[reduce_lr])

In [32]:
our_custom_cnn = chest_x_ray_cnn()

171446536/171446536 [==============================] - 6s 0us/step


In [33]:
our_custom_cnn.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1024, 1024, 3)]      0         []                            
                                                                                                  
 vgg16 (Functional)          (None, 32, 32, 512)          1471468   ['input_1[0][0]']             
                                                          8                                       
                                                                                                  
 vgg19 (Functional)          (None, 32, 32, 512)          2002438   ['input_1[0][0]']             
                                                          4                                       
                                                                                              

In [34]:
def weighted_bcel(Y_train_mb,Y_pred_mb):
    
    cardinality_p = np.count_nonzero(Y_train_mb)
    cardinality_n = (Y_train_mb.shape[0]*Y_train_mb.shape[1]) - cardinality_p
    
    epsilon = 1e-7
    beta_p = (cardinality_p + cardinality_n + epsilon)/(cardinality_p + epsilon)
    beta_n = (cardinality_p + cardinality_n + epsilon)/(cardinality_n + epsilon)
    
    Y_pred_mb = tf.clip_by_value(Y_pred_mb, epsilon, 1.0 - epsilon)
    return -tf.reduce_mean(tf.reduce_mean(beta_p*Y_train_mb*tf.math.log(Y_pred_mb) \
                          + beta_n*(1-Y_train_mb)*tf.math.log(1-Y_pred_mb),axis=0))

In [35]:
def compute_performance_metrics(Y_pred,Y_true,thresh):
    
    Y_pred = Y_pred > thresh
    confusion_matrix = multilabel_confusion_matrix(y_true=Y_true,y_pred=Y_pred)
    summed_confusion_matrix = np.sum(confusion_matrix,axis=0)
    
    tp = summed_confusion_matrix[0,0]
    tn = summed_confusion_matrix[1,1]
    fp = summed_confusion_matrix[0,1]
    fn = summed_confusion_matrix[1,0]
    
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    
    return accuracy,precision,recall    

In [36]:
def predicted_center_pixel_cords(heatmaps):
    
    argmax = list()
    for single_heatmap in heatmaps:
        
        heatmap_channels = cv2.split(single_heatmap)
        argmax_per_heatmap = list()
    
        for single_heatmap_channel in heatmap_channels:    
            single_channel_max_loc = list(np.unravel_index(np.argmax(single_heatmap_channel),single_heatmap_channel.shape))
            argmax_per_heatmap.append(single_channel_max_loc)
        
        argmax.append(np.array(argmax_per_heatmap)/np.array([[7,7]])*224)
        
    return np.array(argmax)

In [37]:
conv_base = Model(inputs=[our_custom_cnn.input],outputs=[our_custom_cnn.layers[5].output])

In [38]:
train_data = training_data_all_info[0:10]

In [39]:
train_data.head()

,Image Index,Finding Labels,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Image Path
0,00000001_000.png,[Cardiomegaly],PA,2682,2749,0.143,0.143,/kaggle/input/data/images_001/images/00000001_...
1,00000001_001.png,"[Cardiomegaly, Emphysema]",PA,2894,2729,0.143,0.143,/kaggle/input/data/images_001/images/00000001_...
2,00000001_002.png,"[Cardiomegaly, Effusion]",PA,2500,2048,0.168,0.168,/kaggle/input/data/images_001/images/00000001_...
3,00000002_000.png,[No Finding],PA,2500,2048,0.171,0.171,/kaggle/input/data/images_001/images/00000002_...
4,00000003_000.png,[Hernia],PA,2582,2991,0.143,0.143,/kaggle/input/data/images_001/images/00000003_...


In [40]:
mhe_Ytrain = multi_hot_encoded_Y_train[0:10]

In [41]:
mhe_Ytrain[0:2]

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.]])

In [42]:
from tensorflow.keras.callbacks import LearningRateScheduler

In [43]:
def lr_scheduler(epoch, lr):
    if epoch < 2:
        return lr  # Keep the initial learning rate for the first 10 epochs
    else:
        return lr * tf.math.exp(-0.1)  # Exponential decay with a decay rate of 0.1 every epoch after the 10th epoch


optimizer = tf.keras.optimizers.RMSprop(lr=0.001)
our_custom_cnn.compile(optimizer=optimizer, loss=weighted_bcel, metrics=['accuracy'])
lr_scheduler_callback = LearningRateScheduler(lr_scheduler)


num_epochs = 4
time_steps = 0
mb_size = 2

for epoch in range(num_epochs):
    for X_train_mb,Y_train_mb in custom_training_data_generator(train_data,mhe_Ytrain,mb_size):
        
        with tf.GradientTape() as tape:
            
            Y_pred_mb = our_custom_cnn(X_train_mb)
            w_bcel_value = weighted_bcel(Y_train_mb,Y_pred_mb)
            
        gradients = tape.gradient(w_bcel_value,our_custom_cnn.trainable_weights)
        optimizer.apply_gradients(zip(gradients,our_custom_cnn.trainable_weights))
        
        train_acc,train_pre,train_rec = compute_performance_metrics(Y_pred_mb,Y_train_mb,0.5)
        
        time_steps += 1
            
        print("\n\nEpoch # {}, Time Step # {}".format(epoch,time_steps))
        print("WBCEL Value = {}, Training Accuracy = {}, Training Precision = {}, Training Recall = {}".format(w_bcel_value,
                                                                                                              train_acc,
                                                                                                              train_pre,
                                                                                                              train_rec))
        conv_base_out = conv_base(X_train_mb).numpy()
        cls_head_params = our_custom_cnn.layers[-1].weights[0].numpy()
        
        heatmaps = np.matmul(conv_base_out,cls_head_params)
        pred_center_pix_loc = predicted_center_pixel_cords(heatmaps)
        
        print("Epoch # {}, Time Step # {}, Predicted Location of Each Disease for Chest X-rays in Mini Batch:".format(epoch,
                                                                                                                     time_steps))
        print(pred_center_pix_loc)



Epoch # 0, Time Step # 1
WBCEL Value = 22.6063232421875, Training Accuracy = 0.4642857142857143, Training Precision = 0.48148148148148145, Training Recall = 0.9285714285714286
Epoch # 0, Time Step # 1, Predicted Location of Each Disease for Chest X-rays in Mini Batch:
[[[  0. 768.]
  [  0. 928.]
  [  0. 832.]
  [  0. 800.]
  [ 32. 992.]
  [ 64. 768.]
  [ 64. 960.]
  [ 64. 768.]
  [  0. 416.]
  [ 96.   0.]
  [  0. 256.]
  [  0. 992.]
  [  0. 512.]
  [608.  96.]]

 [[512. 992.]
  [992. 928.]
  [992. 224.]
  [  0. 512.]
  [736. 448.]
  [ 96. 896.]
  [992. 992.]
  [704. 224.]
  [672. 640.]
  [448. 128.]
  [416. 960.]
  [160.  96.]
  [992. 864.]
  [608. 864.]]]


Epoch # 0, Time Step # 2
WBCEL Value = 19.22898292541504, Training Accuracy = 0.6785714285714286, Training Precision = 0.76, Training Recall = 0.8636363636363636
Epoch # 0, Time Step # 2, Predicted Location of Each Disease for Chest X-rays in Mini Batch:
[[[352. 480.]
  [192.   0.]
  [992. 992.]
  [352.   0.]
  [  0. 960.]
  [960

In [44]:
test_data = testing_data_all_info[0:4]

In [45]:
mhe_Ytest = multi_hot_encoded_Y_test[0:4]

In [46]:
# print("\n\n\n\n")

# thresh_probability = np.arange(start=0.3,stop=0.5,step=0.1)
# fold_size = 2
# K = testing_data_all_info.shape[0]//fold_size

# for fold_idx in range(K):
#     for p_thresh in thresh_probability:
        
#         cv_data = pd.concat([test_data[0:(fold_idx*fold_size)],
#                              test_data[(fold_idx+1)*fold_size:]],axis=0)
        
#         testing_data = test_data[(fold_idx*fold_size):(fold_idx+1)*fold_size]
        
#         multi_hot_encoded_Y_cv = np.concatenate((mhe_Ytest[0:(fold_idx*fold_size)],
#                                                 mhe_Ytest[(fold_idx+1)*fold_size:]),
#                                                 axis=0)
        
#         multi_hot_encoded_Y_test = mhe_Ytest[(fold_idx*fold_size):(fold_idx+1)*fold_size]
        
#         X_cv, Y_cv, rescaled_centroid_cv = custom_testing_data_generator(cv_data,multi_hot_encoded_Y_cv,
#                                                                          cv_data.shape[0])
        
#         X_test, Y_test, rescaled_centroid_test = custom_testing_data_generator(testing_data,
#                                                                                multi_hot_encoded_Y_test,
#                                                                                testing_data.shape[0])
        
#         Y_pred_cv = our_custom_cnn(X_cv)
#         cv_acc,cv_pre,cv_rec = compute_performance_metrics(Y_pred_cv,Y_cv,p_thresh)
        
#         cv_conv_base_out = conv_base(X_cv).numpy()
#         cv_heatmaps = np.matmul(cv_conv_base_out,cls_head_weights)
        
#         Y_pred_test = our_custom_cnn(X_test)
#         test_acc,test_pre,test_rec = compute_performance_metrics(Y_pred_test,Y_test,p_thresh)
        
#         test_conv_base_out = conv_base(X_test).numpy()
#         test_heatmaps = np.matmul(test_conv_base_out,cls_head_weights)
        
#         print("Threshold Probability: {}, CV Acc: {}, CV Prec: {}, CV Rec: {}".format(p_thresh,cv_acc,
#                                                                                             cv_pre,
#                                                                                             cv_rec))
        
#         print("Threshold Probability: {}, Test Acc: {}, Test Prec: {}, Test Rec: {}".format(p_thresh,
#                                                                                            test_acc,
#                                                                                            test_pre,
#                                                                                            test_rec))

In [47]:
x_test, y_test, z_test = custom_testing_data_generator(test_data, mhe_Ytest, 2 ).__next__()

In [48]:
x_test.shape

(2, 1024, 1024, 3)

In [49]:
p_thresh = 0.3
Y_pred_test = our_custom_cnn(x_test)
test_acc,test_pre,test_rec = compute_performance_metrics(Y_pred_test,y_test,p_thresh)
test_conv_base_out = conv_base(x_test).numpy()
test_heatmaps = np.matmul(test_conv_base_out,cls_head_params)
print("Threshold Probability: {}, Test Acc: {}, Test Prec: {}, Test Rec: {}".format(p_thresh,
                                                                                           test_acc,
                                                                                           test_pre,
                                                                                           test_rec))

Threshold Probability: 0.3, Test Acc: 0.8571428571428571, Test Prec: 1.0, Test Rec: 0.8571428571428571
